This Version looks up the Publication Spreadsheet to pull data direct from intake form

In [164]:
# Install packages as needed
#%pip install openpyxl
#%pip install pandas 
#%pip install plotly
#%pip install nbformat
#%pip install --upgrade ipython

In [165]:
#import libraries
import pandas as pd
import json
import os
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import locale
import numpy as np
import re
import plotly.io as pio

Extract

In [166]:
###EXTRACT###
# Read Transaction Excel data into a Pandas dataframe
file = r'C:\Users\DIM6\OneDrive - PGE\[Work] [Shared]\CXI Projects\20200122 03 CC Fact Sheet\2022\2022 Customer Transactions Map - Publication Sheets.xlsx'
sheet_name = 'BarGraphs'

# Read the specific sheet from the Excel file
df_raw = pd.read_excel(file, sheet_name=sheet_name)

print(df_raw.head())

  Root Level Level 1 Level 2a Level 2  Value  TableBlank?  Level
0        NaN     NaN      NaN     NaN    NaN        False    NaN
1        NaN     NaN      NaN     NaN    NaN        False    NaN
2        NaN     NaN      NaN     NaN    NaN        False    NaN
3        NaN     NaN      NaN     NaN    NaN        False    NaN
4        NaN     NaN      NaN     NaN    NaN        False    NaN


In [167]:
#Column Headers
Root = 'Root Level'
L1 = "Level 1" 
L2 = "Level 2"
Value = "Value"
Cat_Level = "Level"

Transform

In [168]:
#Remove columns not needed for barchart
df = df_raw[[Root, L1, L2, Value, Cat_Level]]

# convert cells in column "Value" to numeric values
df[Value] = pd.to_numeric(df[Value], errors='coerce')

# Remove rows with all NaN values
df = df.dropna(how='all')

# Filter out rows where "Level" is 0
df = df[df[Cat_Level] != 0]

# Filter out rows with NaN or blank values in the "Value" column
df = df[(df[Value].notna()) & (df[Value] != '')]

# multiply the dataframe by 1 million to return to nominal units (from Million to 0)
df[Value] = df[Value].mul(1000000)

 #convert the float columns to integers
df[Value] = df[Value].astype(int)

## Replace NaN with None for plotly sunburst chart
df = df.where(pd.notna(df), "")

# Remove "-" and whitespace
df[L1] = df[L1].str.replace(r"\s*-\s*", "").str.strip().astype(str)
df[L2] = df[L2].str.replace(r"\s*-\s*", "").str.strip().astype(str)

# Splice a new df
df = df.copy()

print(df.head())

          Root Level   Level 1   Level 2     Value  Level
43               NPC  Payments  Payments   1770000    1.0
49              Mail  Payments  Payments  11200000    1.0
58  Other Electronic  Payments  Payments  10393000    1.0
59  Other Electronic  Payments  Payments   2735000    1.0
60  Other Electronic  Payments  Payments    637000    1.0


C:\Users\DIM6\AppData\Local\Temp\1\ipykernel_10492\2265567015.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\DIM6\AppData\Local\Temp\1\ipykernel_10492\2265567015.py:26: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\DIM6\AppData\Local\Temp\1\ipykernel_10492\2265567015.py:27: FutureWarning:

The default value of regex will change from True to False in a future version.



In [169]:
# Convert dataframe to a hierarchical dictionary
def make_hierarchy_dict(df, path=[], value_col=Value, threshold=10):
    if len(path) == df.columns.nlevels - 1:
        total = int(df[value_col].sum())
        if total < threshold:
            return None
        else:
            return total
    else:
        level = df.columns[len(path)]
        subdict = {}
        for key in df[level].dropna().unique():
            subdf = df[df[level] == key].drop(columns=[level])
            subhierarchy_dict = make_hierarchy_dict(subdf, path + [key], value_col, threshold)
            if subhierarchy_dict is not None:
                subdict[key] = subhierarchy_dict
        if len(subdict) == 0:
            return None
        else:
            return subdict

hierarchy_dict = make_hierarchy_dict(df.set_index(list(df.columns[:3])), value_col=Value, threshold=10)

print(hierarchy_dict)

110216274


In [170]:
# Convert hierarchical dictionary to Plotly-compatible JSON
def make_plotly_sunburst(hierarchy_dict, name=""):
    if isinstance(hierarchy_dict, dict):
        children = []
        for key, value in hierarchy_dict.items():
            children.append(make_plotly_sunburst(value, key))
        return {
            "name": name,
            "children": children
        }
    else:
        return {
            "label": name,
            "value": hierarchy_dict
        }

In [171]:
# Write JSON data to a file
plotly_data = make_plotly_sunburst(hierarchy_dict)

with open("plotly_data.json", "w") as f:
    json.dump(plotly_data, f)

    print(plotly_data)

{'label': '', 'value': 110216274}


In [172]:
# Write dataframe to an Excel file for QC
with pd.ExcelWriter('dataframe-bargraph-tmp.xlsx') as writer:
    df.to_excel(writer, sheet_name='dataframe-bargraph', index=False)

Load

In [173]:
#Group the data by Root Level and sum the values
grouped_data = df.groupby([Root])[Value].sum().reset_index()

# Sort the grouped_data by Value column in descending order
grouped_data = grouped_data.sort_values(by=Value, ascending=False)

# Create a dictionary to map the Root Level to its corresponding color
color_map = {'Web': 'orange', 'Other Electronic': 'gray', 'Mail': 'burlywood', 'Call Center IVR': 'deepskyblue','Call Center Other':'lightblue', \
              'Call Center - Live Agent':'Turquoise', 'NPC': 'gold'}

# Create the bar chart trace
bar_trace = go.Bar(
    x=grouped_data[Root],
    y=grouped_data[Value],
    marker=dict(color=[color_map.get(root_level, 'grey') for root_level in grouped_data[Root]]),
    text=[f"{value:,}" for value in grouped_data[Value]],
    textposition='auto'
)

# Create the bar chart layout
bar_layout = go.Layout(
    title='Total Customer Interactions by Transaction Channel',
    xaxis=dict(title='Channel'),
    yaxis=dict(title='Total Interactions'),
    legend=dict(title=dict(text='Transaction Channels'))
)

# Create the bar chart figure
bar_fig_root = go.Figure(data=[bar_trace], layout=bar_layout)

# Display the bar chart
bar_fig_root.show()


In [174]:
# Export the chart as an HTML file
html = ".html"
rootbarchart = 'Trxn_root_barchart'
pyo.plot(bar_fig_root, filename=(rootbarchart + html), auto_open=True)

# Generate the chart as an ASPX file
aspx = '.aspx'
pio.write_html(bar_fig_root, file=(rootbarchart + aspx), auto_open=False)

In [175]:
# Set the locale to use comma separators for thousands
locale.setlocale(locale.LC_ALL, '')

# Group the data by Root Level and Level 1 and sum the values
grouped_data = df.groupby([Root, L1])[Value].sum().reset_index()

# Sort the grouped_data by Value column in descending order
grouped_data = grouped_data.sort_values(by=Value, ascending=False)

# Create a dictionary to map the Root Level to its corresponding color
color_map = {'Web': 'orange', 'Other Electronic': 'gray', 'Mail': 'burlywood', 'Call Center - Live Agent': 'turquoise', 'Call Center IVR': 'deepskyblue', 'Call Center Other':'lightblue', 'NPC': 'gold'}


# Create a list to store the bar chart traces
bar_traces = []

# Loop through each Root Level and create a stacked bar chart trace for its corresponding Level 1 data
for root_level in grouped_data[Root].unique():
    temp_df = grouped_data[grouped_data[Root] == root_level]
    values = temp_df[Value].values
    formatted_values = [locale.format_string("%d", val, grouping=True) for val in values] # Format values with comma separators
    bar_trace = go.Bar(
        x=temp_df[L1],
        y=values,
        text=formatted_values, # Set the formatted values as the text labels
        textposition='auto', # Position the text labels automatically above the bars
        name=root_level,
        marker=dict(color=color_map[root_level])
    )
    bar_traces.append(bar_trace)

# Create the stacked bar chart layout
bar_layout = go.Layout(
    title='Total Customer Interactions by Service',
    xaxis=dict(title='Service'),
    yaxis=dict(title='Total Interactions'),
    barmode='stack',
    height=900,
    legend=dict(title=dict(text='Transaction Channels'))
)

# Create the stacked bar chart figure
bar_fig_root_L1 = go.Figure(data=bar_traces, layout=bar_layout)

# Display the stacked bar chart
bar_fig_root_L1.show()


In [176]:
# Export the chart as an HTML file
html = ".html"
rootbarchartL1 = 'Trxn_root_barchart_L1'

pyo.plot(bar_fig_root_L1, filename=(rootbarchartL1 + html), auto_open=True)

# Generate the chart as an ASPX file
aspx = '.aspx'
pio.write_html(bar_fig_root_L1, file=(rootbarchartL1 + aspx), auto_open=False)

In [177]:
# Set the locale to use comma separators for thousands
locale.setlocale(locale.LC_ALL, '')

# Group the data by Root Level and Level 2 and sum the values
grouped_data = df.groupby([Root, L2])[Value].sum().reset_index()

# Sort the grouped_data by Value column in descending order
grouped_data = grouped_data.sort_values(by=Value, ascending=False) # Sort by descending order

# Create a dictionary to map the Root Level to its corresponding color
color_map = {'Web': 'orange', 'Other Electronic': 'gray', 'Mail': 'burlywood',   'Call Center - Live Agent': 'turquoise', 'Call Center IVR': 'deepskyblue', 'Call Center Other':'lightblue', 'NPC': 'gold'}



# Create a list to store the bar chart traces
bar_traces = []

# Loop through each Root Level and create a stacked bar chart trace for its corresponding Level 2 data
for root_level in grouped_data[Root].unique():
    temp_df = grouped_data[grouped_data[Root] == root_level]
    values = temp_df[Value].values
    formatted_values = [locale.format_string("%d", val, grouping=True) for val in values] # Format values with comma separators
    y_values = temp_df[L2].values
    bar_trace = go.Bar(
        y=y_values,
        x=values,
        text=formatted_values,
        textposition='auto',
        name=root_level,
        orientation='h',
        marker=dict(color=color_map[root_level])
    )
    bar_traces.append(bar_trace)

# Create the stacked bar chart layout
bar_layout = go.Layout(
    title='Total Customer Interactions by Service - Breakdown',
    xaxis=dict(title='Total Interactions'),
    yaxis=dict(title='Service Detail (from lowest to highest)'),
    barmode='stack',
    height=1600,
    legend=dict(title=dict(text='Transaction Channels'))
)

# Create the stacked bar chart figure
bar_fig_L2 = go.Figure(data=bar_traces, layout=bar_layout)

# Display the stacked bar chart
bar_fig_L2.show()


In [178]:
# Export the chart as an HTML file
html = ".html"
rootbarchartL2 = 'Trxn_root_barchart_L2'

pyo.plot(bar_fig_L2, filename=(rootbarchartL2 + html), auto_open=True)

# Generate the chart as an ASPX file
aspx = '.aspx'
pio.write_html(bar_fig_L2, file=(rootbarchartL2 + aspx), auto_open=False)